<a href="https://colab.research.google.com/github/Kartikey1101/secure_KNN/blob/main/CAD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

Saving netlist.edif to netlist.edif


In [81]:
#Code snippet to remove input and output ports from the .edif file
def extract_ports_from_design(edif_content):
    # Initialize variables to store results
    in_design_library = False
    ports = []

    # Split the content by lines for easier processing
    lines = edif_content.splitlines()

    for line in lines:
        # Check for the start of the DESIGN library section
        if "(library DESIGN" in line:
            in_design_library = True

        # If inside the DESIGN library, look for port definitions
        if in_design_library:
            line = line.strip()

            # Check if the line starts with "(port" and capture the port name and direction
            if line.startswith("(port"):
                parts = line.split()
                port_name = parts[1]
                direction = parts[-1].strip('()')
                ports.append((port_name, direction))

            # Check for the closing brackets indicating the end of the DESIGN section
            if line.startswith(")"):
                break

    return ports

def extract_instances_from_design(edif_content):
    in_design_library = False
    instances = []
    gate_data = []
    # Split the content by lines for easier processing
    lines = edif_content.splitlines()

    for line in lines:
        # Check for the start of the DESIGN library section
        if "(library DESIGN" in line:
            in_design_library = True

        # If inside the DESIGN library, look for instance definitions
        if in_design_library:
            line = line.strip()

        # Check if the line starts with "(instance" and capture the instance ID and gate type
        if line.startswith("(instance") and line.startswith("(rename",10,20):
            parts = line.split()
            instance_id = parts[2]
            gate_type = parts[3].split('/')[0].strip('"$')  # Extract gate type before '/'
            instances.append((instance_id, gate_type))

        # Check for the closing brackets indicating the end of the DESIGN section
        if line.startswith(")))"):
            break

    return instances

def extract_nets_from_design(edif_content):
    in_design_library = False
    in_net = False
    in_net2 = False
    out_net = False

    nets = []
    net_data = []
    gate_data = []

    # Split the content by lines for easier processing
    lines = edif_content.splitlines()

    for line in lines:
        # Check for the start of the DESIGN library section
        if "(library DESIGN" in line:
            in_design_library = True

        # If inside the DESIGN library, look for instance definitions
        if in_design_library:
            line = line.strip()

        # Check if the line starts with "(instance" and capture the instance ID and gate type
        if line.startswith("(net") and line.startswith("(rename",5,20):
            parts = line.split()
            net_id = parts[2]
            in_net = True
        elif line.startswith("(net") and not line.startswith("(rename",5,20): #for PI
            parts = line.split()
            net_id = parts[1]
            in_net2 = True
            #print(net_id)


        if in_net and line.startswith("(portRef"):
            parts = line.split()
            if len(parts) >= 4:
              input_to  = (parts[3].strip(')'),parts[1])
              out_net = True
              in_net = False
              #print(input_to)
              continue

        if in_net2 and line.startswith("(portRef"):
            parts = line.split()
            if len(parts) >= 4:
              input_to  = (parts[3].strip(')'),parts[1])
              out_net = True
              in_net2 = False
              #print(input_to)
              continue



        if out_net and line.startswith("(portRef"):
            parts = line.split()
            if len(parts) >= 4:
              output_from = (parts[3].strip(')'),parts[1])
              nets.append((net_id, output_from, input_to))
            elif len(parts) >= 1:
              output_from = (parts[1].strip(')'))
              #print(output_from)
              nets.append((net_id, output_from, input_to))

        # Check for the closing brackets indicating the end of the DESIGN section
        if line.startswith("))"):
           out_net = False

    return nets


In [82]:
all_inputs = [] #pi
all_outputs = [] #po
gate_data = [] #gate data [id0003 or,id004 and]
gate_id = [] #gate id [id0003,id0004]
gate_typee = [] #gate type [or , and ]

net_idd = [] # all nets
fromm = [] # from gate id of
fromm_section = [] # contains A,B,Y of gates
too = []
too_section = []

file_path = 'netlist.edif'

# Read the Verilog code from the file
with open(file_path, 'r') as file:
    edif_content = file.read()

ports = extract_ports_from_design(edif_content)
print("Extracted Ports and Directions:")
for port, direction in ports:
    if(direction == 'INPUT'):
      all_inputs.append(port)
    elif(direction == 'OUTPUT'):
      all_outputs.append(port)
    print(f"Port: {port}, Direction: {direction}")

instances = extract_instances_from_design(edif_content)
print("Extracted Instances and Gate Types:")
for instance_id, gate_type in instances:
    gate_id.append(instance_id)
    gate_typee.append(gate_type)
    print(instance_id,",",gate_type)

print(".................")
# print(instances.type())
print(gate_data)
nets = extract_nets_from_design(edif_content)
print("Extracted nets and their connections:")
for net_id, output_from, input_to in nets:
        # net_idd.append(net_id)
        # fromm.append(output_from[0])
        # fromm_section.append(output_from[1])
        # if (type(input_to) == tuple):
        #   too.append(input_to[0])
        #   too_section.append(input_to[1])
        # else:
        #   too.append(input_to)
        #   too_section.append(0)
        print("----------------------------------------")
        print(type(input_to))
        print(net_id , "," , output_from ,"," , input_to)
        # print(net_id  ,"," , input_to ,"," , output_from , )

Extracted Ports and Directions:
Port: zt, Direction: OUTPUT
Port: d, Direction: INPUT
Port: c, Direction: INPUT
Port: b, Direction: INPUT
Port: a, Direction: INPUT
Extracted Instances and Gate Types:
id00003 , or
id00004 , and
id00005 , and
.................
[]
Extracted nets and their connections:
----------------------------------------
<class 'tuple'>
id00006 , ('id00004', 'Y') , ('id00003', 'B')
----------------------------------------
<class 'tuple'>
a , a , ('id00005', 'A')
----------------------------------------
<class 'tuple'>
c , c , ('id00004', 'A')
----------------------------------------
<class 'tuple'>
b , b , ('id00005', 'B')
----------------------------------------
<class 'tuple'>
d , d , ('id00004', 'B')
----------------------------------------
<class 'tuple'>
id00007 , ('id00005', 'Y') , ('id00003', 'A')
----------------------------------------
<class 'tuple'>
id00008 , zt , ('id00003', 'Y')


In [83]:
import re
from collections import defaultdict

# Define a Node class for structured representation
class Node:
    def __init__(self, name, operation, inputs, output, level=None, inversion_parity=0, num_inputs=0, num_outputs=0, value=None):
        self.name = name
        self.operation = operation
        self.inputs = inputs
        self.output = output
        self.level = level
        self.inversion_parity = inversion_parity
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.value = value

def parse_verilog(file_path):
    nodes = []
    output_references = defaultdict(int)  # Track how many times each output is used as an input
    primary_inputs = set()
    gate_nodes = []  # List to store gates and their info for later processing
    node_dependencies = {}  # Track dependencies to calculate levels
    primary_inputs = []
    final_output = []

    with open(file_path, 'r') as f:
        edif_content = f.read()

    ports = extract_ports_from_design(edif_content)
    # print("Extracted Ports and Directions:")
    for port, direction in ports:
        if(direction == 'INPUT'):
          primary_inputs.append(port)
        elif(direction == 'OUTPUT'):
          final_output.append(port)
        print(f"Port: {port}, Direction: {direction}")

    # Adjusted regex pattern to capture gates (AND, OR, NOT, NAND, NOR, XOR, XNOR) with their connections
    # gate_pattern = r"(and|or|not|nand|nor|xor|xnor)\s+(\w+)\s*\((\w+)(?:,\s*([\w\s,]+))?\);"

    instances = extract_instances_from_design(edif_content)
    print("Extracted Instances and Gate Types:")
    for instance_id, gate_type in instances:
        name = instance_id
        operation = gate_type

        # Determine inversion parity: 1 for NOT, NAND, NOR, XNOR; 0 for others
        inversion_parity = 1 if operation =='not' else 0

        # Create and add the gate node (level is calculated later)
        node = Node(name=name, operation=operation, inputs=[], output= [],
                    level=None, inversion_parity=inversion_parity, num_inputs=None, num_outputs=0, value=None)
        gate_nodes.append(node)
        # node_dependencies[output] = inputs



    nets = extract_nets_from_design(edif_content)
    print("Extracted nets and their connections:")

    def update_node(gate_nodes, node_name, value, operator):
        # print(node_name)
        for node in gate_nodes:
            if node.name == node_name:
                if (value == 'A' or value == 'B'):
                  # print("input")
                  # print(operator)
                  node.inputs.append(operator)
                if (value == 'Y' ):
                  # print("output")
                  # print(operator)
                  node.output = operator



    for net_id, output_from,input_to in nets:


            if (type(output_from) == tuple):
               update_node(gate_nodes,input_to[0],input_to[1], net_id)
               update_node(gate_nodes,output_from[0],output_from[1],net_id)
            else:
               if (net_id == output_from ):
                   update_node(gate_nodes,output_from,input_to[1],net_id)
               else:
                   update_node(gate_nodes,input_to[0],input_to[1], output_from)

    for node in gate_nodes:
       print(f"Node {node.name}:")
       print(f"  Operation: {node.operation.upper()}")
       print("-------------------------------------------------")
       print(f"  Inputs: {', '.join(node.inputs)}")


       print(f"  Output: {node.output}")
       print(f"  Level: {node.level}")
       print(f"  Number of Inputs: {node.num_inputs}")
       print(f"  Number of Outputs: {node.num_outputs}")
       print(f"  Inversion Parity: {node.inversion_parity}")
       print(f"  Current Value: {node.value}")
       print()

In [84]:
parse_verilog('netlist.edif')


Port: zt, Direction: OUTPUT
Port: d, Direction: INPUT
Port: c, Direction: INPUT
Port: b, Direction: INPUT
Port: a, Direction: INPUT
Extracted Instances and Gate Types:
Extracted nets and their connections:
Node id00003:
  Operation: OR
-------------------------------------------------
  Inputs: id00006, id00007
  Output: zt
  Level: None
  Number of Inputs: None
  Number of Outputs: 0
  Inversion Parity: 0
  Current Value: None

Node id00004:
  Operation: AND
-------------------------------------------------
  Inputs: 
  Output: id00006
  Level: None
  Number of Inputs: None
  Number of Outputs: 0
  Inversion Parity: 0
  Current Value: None

Node id00005:
  Operation: AND
-------------------------------------------------
  Inputs: 
  Output: id00007
  Level: None
  Number of Inputs: None
  Number of Outputs: 0
  Inversion Parity: 0
  Current Value: None



In [85]:
import re
from collections import defaultdict

# Define a Node class for structured representation
class Node:
    def __init__(self, name, operation, inputs, output, level=None, inversion_parity=0, num_inputs=0, num_outputs=0, value=None):
        self.name = name
        self.operation = operation
        self.inputs = inputs
        self.output = output
        self.level = level
        self.inversion_parity = inversion_parity
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.value = value

def parse_verilog(file_path):
    nodes = []
    output_references = defaultdict(int)  # Track how many times each output is used as an input
    primary_inputs = set()
    gate_nodes = []  # List to store gates and their info for later processing
    node_dependencies = {}  # Track dependencies to calculate levels
    primary_inputs = []
    final_output = []

    with open(file_path, 'r') as f:
        edif_content = f.read()

    ports = extract_ports_from_design(edif_content)
    # print("Extracted Ports and Directions:")
    for port, direction in ports:
        if(direction == 'INPUT'):
          primary_inputs.append(port)
        elif(direction == 'OUTPUT'):
          final_output.append(port)
        print(f"Port: {port}, Direction: {direction}")

    # Adjusted regex pattern to capture gates (AND, OR, NOT, NAND, NOR, XOR, XNOR) with their connections
    # gate_pattern = r"(and|or|not|nand|nor|xor|xnor)\s+(\w+)\s*\((\w+)(?:,\s*([\w\s,]+))?\);"

    instances = extract_instances_from_design(edif_content)
    print("Extracted Instances and Gate Types:")
    for instance_id, gate_type in instances:
        name = instance_id
        operation = gate_type

        # Determine inversion parity: 1 for NOT, NAND, NOR, XNOR; 0 for others
        inversion_parity = 1 if operation =='not' else 0

        # Create and add the gate node (level is calculated later)
        node = Node(name=name, operation=operation, inputs=[], output= None,
                    level=None, inversion_parity=inversion_parity, num_inputs=None, num_outputs=0, value=None)
        gate_nodes.append(node)
        # node_dependencies[output] = inputs



    nets = extract_nets_from_design(edif_content)
    print("Extracted nets and their connections:")

    def update_node(gate_nodes, node_name, value, operator):
        # print(node_name)
        for node in gate_nodes:
            if node.name == node_name:
                if (value == 'A' or value == 'B'):
                  # print("input")
                  # print(operator)
                  node.inputs.append(operator)
                if (value == 'Y' ):
                  # print("output")
                  # print(operator)
                  node.output = operator



    for net_id, output_from,input_to in nets:


            if (type(output_from) == tuple):
               update_node(gate_nodes,input_to[0],input_to[1], net_id)
               update_node(gate_nodes,output_from[0],output_from[1],net_id)
            else:
               if (net_id == output_from ):
                   update_node(gate_nodes,output_from,input_to[1],net_id)
               else:
                   update_node(gate_nodes,input_to[0],input_to[1], output_from)


# First pass: Create nodes and store gates without calculating levels yet
    for node in gate_nodes:

         # Number of inputs
         node.num_inputs = len(node.inputs)

         # Track output references for num_outputs calculation
         for inp in node.inputs:
             output_references[inp] += 1
             # Example: If node.inputs is ['A', 'B'], output_references['A'] and output_references['B'] are incremented.

         # Track dependencies for level calculation
         node_dependencies[node.output] = node.inputs


    # Second pass: Resolve levels for each node
    def resolve_level(node):
        """Recursively resolve and calculate levels."""
        if node.level is not None:
            return node.level  # Already resolved

        input_levels = []
        for inp in node.inputs:
            if inp in primary_inputs:
                input_levels.append(0)  # Primary inputs have level 0
            else:
                # Find the node that produces this input
                dep_node = next((n for n in gate_nodes if n.output == inp), None)
                if dep_node:
                    input_levels.append(resolve_level(dep_node))
                else:
                    input_levels.append(0)  # Assume primary input if not found

        node.level = max(input_levels) + 1 if input_levels else 1
        return node.level

    # Resolve levels for all nodes
    for node in gate_nodes:
        resolve_level(node)

    # Third pass: Calculate num_outputs for each node
    for node in gate_nodes:
        node.num_outputs = output_references[node.output]

    print("Primary Inputs:", primary_inputs)
    print("Final Output:", final_output)

    # print("Primary Inputs:", inputs_match)
    # print("Final Output:", primary_outputs)


    # Print the nodes with their levels and number of outputs, inversion parity, number of inputs, and current value
    for node in gate_nodes:
        print(f"Node {node.name}:")
        print(f"  Operation: {node.operation.upper()}")
        print(f"  Inputs: {', '.join(node.inputs)}")
        print(f"  Output: {node.output}")
        print(f"  Level: {node.level}")
        print(f"  Number of Inputs: {node.num_inputs}")
        print(f"  Number of Outputs: {node.num_outputs}")
        print(f"  Inversion Parity: {node.inversion_parity}")
        print(f"  Current Value: {node.value}")
        print()

    return primary_inputs, final_output, gate_nodes

# Call the function with the Verilog file
parse_verilog('netlist.edif')


Port: zt, Direction: OUTPUT
Port: d, Direction: INPUT
Port: c, Direction: INPUT
Port: b, Direction: INPUT
Port: a, Direction: INPUT
Extracted Instances and Gate Types:
Extracted nets and their connections:
Primary Inputs: ['d', 'c', 'b', 'a']
Final Output: ['zt']
Node id00003:
  Operation: OR
  Inputs: id00006, id00007
  Output: zt
  Level: 2
  Number of Inputs: 2
  Number of Outputs: 0
  Inversion Parity: 0
  Current Value: None

Node id00004:
  Operation: AND
  Inputs: 
  Output: id00006
  Level: 1
  Number of Inputs: 0
  Number of Outputs: 1
  Inversion Parity: 0
  Current Value: None

Node id00005:
  Operation: AND
  Inputs: 
  Output: id00007
  Level: 1
  Number of Inputs: 0
  Number of Outputs: 1
  Inversion Parity: 0
  Current Value: None



(['d', 'c', 'b', 'a'],
 ['zt'],
  <__main__.Node at 0x7d868427bfd0>])

In [86]:

def simulate_circuit(nodes, primary_inputs, final_outputs):
    # Prompt the user for binary input values
    input_values = {}
    print("Enter binary values (0 or 1) for the following primary inputs:")
    for inp in primary_inputs:
        while True:
            try:
                value = int(input(f"{inp}: "))
                if value not in (0, 1):
                    raise ValueError("Value must be 0 or 1.")
                input_values[inp] = value
                break
            except ValueError as e:
                print(e)

    # Display initial input values
    print("Primary Input Values:")
    for inp, val in input_values.items():
        print(f"{inp} = {val}")

    def and_gate(inputs):
        return int(all(inputs))  # Returns 1 if all inputs are 1, otherwise 0

    def or_gate(inputs):
        return int(any(inputs))  # Returns 1 if any input is 1, otherwise 0

    def not_gate(inputs):
        return int(not inputs[0])  # Binary inversion

    def xor_gate(inputs):
        """XOR operation for multiple inputs."""
        result = 0
        for inp in inputs:
            result ^= inp  # XOR accumulates based on the number of 1s
        return result

    def xnor_gate(inputs):
        """XNOR operation for multiple inputs (inverse of XOR)."""
        return int(not xor_gate(inputs))

    def nand_gate(inputs):
        """NAND operation for multiple inputs."""
        result = 1
        for inp in inputs:
            result &= inp
        return int(not result)  # Inverted result of AND

    def nor_gate(inputs):
        """NOR operation for multiple inputs."""
        result = 0
        for inp in inputs:
            result |= inp
        return int(not result)  # Inverted result of OR

    # Simulation results storage
    values = input_values.copy()

    def xor_gate(inputs):
    # ""XOR operation for multiple inputs.""
        result = 0
        for inp in inputs:
          result ^= inp  # XOR accumulates based on the number of 1s
        return result


    # Process nodes level by level
    max_level = max(node.level for node in nodes)
    for level in range(1, max_level + 1):
        print(f"\nProcessing level {level} gates:")
        for node in nodes:
            if node.level == level:
                # Gather the input values for this node
                input_vals = [values[inp] for inp in node.inputs]

                # Simulate the operation based on the gate type
                if node.operation == 'and':
                    node.value = and_gate(input_vals)
                elif node.operation == 'or':
                    node.value = or_gate(input_vals)
                elif node.operation == 'not':
                    node.value = not_gate(input_vals)
                elif node.operation == 'xor':
                    node.value = xor_gate(input_vals)
                elif node.operation == 'xnor':
                    node.value = xnor_gate(input_vals)
                elif node.operation == 'nand':
                    node.value = nand_gate(input_vals)
                elif node.operation == 'nor':
                    node.value = nor_gate(input_vals)

                # Store the output value
                values[node.output] = node.value

                # Display the node processing
                print(f"Node {node.name}: {node.operation}({', '.join(node.inputs)}) = {node.value}")

    # Handle multiple final outputs
    results = {}
    for node in nodes:
        if node.output in final_outputs:
            results[node.output] = values.get(node.output, None)

    if not results:
        print("Error: No final outputs found in the processed nodes.")
    else:
        print("\nFinal Outputs:")
        for output, value in results.items():
            print(f"{output}: {value}")

    return results

# Example usage
file_path = 'netlist.edif'

# Parse the Verilog file and print the results
primary_inputs, final_outputs, nodes = parse_verilog(file_path)
print(f"Primary Inputs: {primary_inputs}")
print(f"Final Outputs: {final_outputs}")

print(".....................")


simulate_circuit(nodes, primary_inputs, final_outputs)

Port: zt, Direction: OUTPUT
Port: d, Direction: INPUT
Port: c, Direction: INPUT
Port: b, Direction: INPUT
Port: a, Direction: INPUT
Extracted Instances and Gate Types:
Extracted nets and their connections:
Primary Inputs: ['d', 'c', 'b', 'a']
Final Output: ['zt']
Node id00003:
  Operation: OR
  Inputs: id00006, id00007
  Output: zt
  Level: 2
  Number of Inputs: 2
  Number of Outputs: 0
  Inversion Parity: 0
  Current Value: None

Node id00004:
  Operation: AND
  Inputs: 
  Output: id00006
  Level: 1
  Number of Inputs: 0
  Number of Outputs: 1
  Inversion Parity: 0
  Current Value: None

Node id00005:
  Operation: AND
  Inputs: 
  Output: id00007
  Level: 1
  Number of Inputs: 0
  Number of Outputs: 1
  Inversion Parity: 0
  Current Value: None

Primary Inputs: ['d', 'c', 'b', 'a']
Final Outputs: ['zt']
.....................
Enter binary values (0 or 1) for the following primary inputs:
d: 1
c: 1
b: 1
a: 1
Primary Input Values:
d = 1
c = 1
b = 1
a = 1

Processing level 1 gates:
Node i

{'zt': 1}